In [4]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = "amazon-dynamo-sosp2007.pdf"
text = extract_text_from_pdf(pdf_path)
print(text[:500])  # Print the first 500 characters to verify

Dynamo: Amazon’s Highly Available Key-value Store 
Giuseppe DeCandia, Deniz Ha storun, Madan Jampani, Guna vardhan Kakulapati,  
Avinash Lakshman, Alex Pilchin, Swam inathan Sivasubramanian, Peter Vosshall  
and Werner Vogels 
Amazon.com 
 
ABSTRACT 
Reliability at massive scale is one of the biggest challenges we 
face at Amazon.com, one of the largest e-commerce operations in 
the world; even the slightest outage has significant financial 
consequences and impacts cust omer trust. The Amazon.c


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Number of characters per chunk
    chunk_overlap=50  # Overlap between chunks
)

chunks = text_splitter.split_text(text)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 208


In [7]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight embedding model
chunk_embeddings = embedding_model.encode(chunks)

In [8]:
import faiss
import numpy as np

# Create a FAISS index
dimension = chunk_embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity search
index.add(np.array(chunk_embeddings))  # Add embeddings to the index

In [9]:
def retrieve_relevant_chunks(question, top_k=3):
    question_embedding = embedding_model.encode([question])
    distances, indices = index.search(question_embedding, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    return relevant_chunks

question = "What is the main topic of the document?"
relevant_chunks = retrieve_relevant_chunks(question)
print("Relevant chunks:", relevant_chunks)

Relevant chunks: ['read/write quorum characteristics. The following are the main \npatterns in which Dynamo is used: \n• Business logic specific reconciliation:  This is a popular use \ncase for Dynamo. Each data object is replicated across multiple nodes. In case of dive rgent versions, the client \napplication performs its ow n reconciliation logic. The \nshopping cart service discussed ear lier is a prime example of \nthis category. Its business l ogic reconciles objects by', 'reads) or storing data at one or more nodes (for writes). Each \nclient request results in the crea tion of a state machine on the node \nthat received the client request. The state machine contains all the \nlogic for identifying the nodes responsible for a key, sending the \nrequests, waiting for response s, potentially doing retries, \nprocessing the replies and packag ing the response to the client. \nEach state machine instance handles exactly one client request.', 'interface; it exposes two operations: g

In [10]:
question = "How do we increase the availability?"
relevant_chunks = retrieve_relevant_chunks(question)

In [11]:
relevant_chunks

['be achieved under which conditions. \nFor systems prone to server and network failures, availability can \nbe increased by using optimistic  replication techniques, where \nchanges are allowed to propagate to replicas in the background, \nand concurrent, disconnected work  is tolerated.  The challenge \nwith this approach is that it can  lead to conflicting changes which \nmust be detected and resolved.  This process of conflict resolution',
 'strict operational requirements on Amazon’s platform in terms of \nperformance, reliabilit y and efficiency, and to support continuous \ngrowth the platform needs to be highly scalable. Reliability is one \nof the most important requirements because even the slightest \noutage has significant financia l consequences and impacts \ncustomer trust. In addition, to  support continuous growth, the \nplatform needs to be highly scalable. One of the lessons our organiza tion has learned from operating',
 'control. In the past, centralized control has 

In [12]:
from transformers import pipeline

# Load a pre-trained question-answering model
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-small")

def generate_answer(question, relevant_chunks):
    context = " ".join(relevant_chunks)
    input_text = f"question: {question} context: {context}"
    answer = qa_pipeline(input_text, max_length=200)[0]['generated_text']
    return answer

answer = generate_answer(question, relevant_chunks)
print("Answer:", answer)

Device set to use mps:0


Answer: optimising replication techniques


In [14]:
question = "How does Dynamodb handle server failure"
relevant_chunks = retrieve_relevant_chunks(question)

In [15]:
answer = generate_answer(question, relevant_chunks)
print("Answer:", answer)

Answer: asynchronous response


In [13]:
device = torch.device("mps")

NameError: name 'torch' is not defined